## SQL | EDA of Google Analytics Data
- [Dillon Myrick 노트북](https://www.kaggle.com/code/dillonmyrick/sql-eda-of-google-analytics-data/notebook)
- [데이터 설명](https://support.google.com/analytics/answer/3437719?hl=en)

In [5]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


### 1. 데이터 확인 
- Bigquery와 connect해 데이터 살펴보기
- [빅쿼리 사용을 위한 라이브러리 설치](https://github.com/googleapis/python-bigquery)
- [사용자 인증 정보 설정](https://cloud.google.com/docs/authentication/provide-credentials-adc?hl=ko#how-to)

In [6]:
# pip install google-cloud-bigquery
# pip install --upgrade google-cloud-bigquery

In [7]:
from google.cloud import bigquery

client = bigquery.Client(project = "prime-dogfish-374600")
dataset_ref = client.dataset('google_analytics_sample', project = 'bigquery-public-data')

dataset = client.get_dataset(dataset_ref)

c:\Users\cyh51\anaconda3\lib\site-packages\google\auth\_default.py:78: UserWarning:

Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 



In [8]:
# dataset 확인
[x.table_id for x in client.list_tables(dataset)][:5]

['ga_sessions_20160801',
 'ga_sessions_20160802',
 'ga_sessions_20160803',
 'ga_sessions_20160804',
 'ga_sessions_20160805']

In [9]:
# pip install db-dtypes
# pip install --upgrade 'google-cloud-bigquery[all]'
# pip install --upgrade google-cloud-bigquery-storage google-cloud-bigquery-datatransfer

In [10]:
# table_referencce 생성
table_ref_20160801 = dataset_ref.table('ga_sessions_20160801')

table_20160801 = client.get_table(table_ref_20160801)

# 데이터 확인
client.list_rows(table_20160801, max_results = 5).to_dataframe()

,visitorId,visitNumber,visitId,visitStartTime,date,totals,trafficSource,device,geoNetwork,customDimensions,hits,fullVisitorId,userId,channelGrouping,socialEngagementType
0,<NA>,1,1470046245,1470046245,20160801,"{'visits': 1, 'hits': 24, 'pageviews': 17, 'ti...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Firefox', 'browserVersion': 'not ...","{'continent': 'Europe', 'subContinent': 'Weste...","[{'index': 4, 'value': 'EMEA'}]","[{'hitNumber': 1, 'time': 0, 'hour': 3, 'minut...",895954260133011192,None,Organic Search,Not Socially Engaged
1,<NA>,1,1470084717,1470084717,20160801,"{'visits': 1, 'hits': 24, 'pageviews': 18, 'ti...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Internet Explorer', 'browserVersi...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 13, 'minu...",0288478011259077136,None,Direct,Not Socially Engaged
2,<NA>,3,1470078988,1470078988,20160801,"{'visits': 1, 'hits': 27, 'pageviews': 17, 'ti...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Safari', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 12, 'minu...",6440789996634275026,None,Organic Search,Not Socially Engaged
3,<NA>,4,1470075581,1470075581,20160801,"{'visits': 1, 'hits': 27, 'pageviews': 19, 'ti...","{'referralPath': '/', 'campaign': '(not set)',...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 11, 'minu...",8520115029387302083,None,Referral,Not Socially Engaged
4,<NA>,30,1470099026,1470099026,20160801,"{'visits': 1, 'hits': 27, 'pageviews': 17, 'ti...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 17, 'minu...",6792260745822342947,None,Organic Search,Not Socially Engaged


In [11]:
# 데이터별로 정리 

def format_schema_field(schema_field, indent = 0):
    indent_str = "  " * indent
    field_info = f"{indent_str}{schema_field.name} ({schema_field.field_type})"

    if schema_field.mode != "NULLABLE":
        field_info += f" - {schema_field.mode}"

    if schema_field.description:
        field_info += f" - {schema_field.description}"

    nested_indent = indent + 2
    if schema_field.field_type == "RECORD":
        for sub_field in schema_field.fields:
            field_info += "\n" + format_schema_field(sub_field, nested_indent)

    return field_info

In [12]:
for i in range(5,11):
    print(format_schema_field(table_20160801.schema[i]))
    print('\n')

totals (RECORD)
    visits (INTEGER)
    hits (INTEGER)
    pageviews (INTEGER)
    timeOnSite (INTEGER)
    bounces (INTEGER)
    transactions (INTEGER)
    transactionRevenue (INTEGER)
    newVisits (INTEGER)
    screenviews (INTEGER)
    uniqueScreenviews (INTEGER)
    timeOnScreen (INTEGER)
    totalTransactionRevenue (INTEGER)
    sessionQualityDim (INTEGER)


trafficSource (RECORD)
    referralPath (STRING)
    campaign (STRING)
    source (STRING)
    medium (STRING)
    keyword (STRING)
    adContent (STRING)
    adwordsClickInfo (RECORD)
        campaignId (INTEGER)
        adGroupId (INTEGER)
        creativeId (INTEGER)
        criteriaId (INTEGER)
        page (INTEGER)
        slot (STRING)
        criteriaParameters (STRING)
        gclId (STRING)
        customerId (INTEGER)
        adNetworkType (STRING)
        targetingCriteria (RECORD)
            boomUserlistId (INTEGER)
        isVideoAd (BOOLEAN)
    isTrueDirect (BOOLEAN)
    campaignCode (STRING)


device (RECOR

## 2. Make Our Goals Concrete
1. 가장 인기 있는 페이지와 이탈률 분석
2. 브라우저 및 디바이스별 세션 수 탐색
3. 사이트 트래픽 유형과 품질 탐색
4. 고객 전환 경로 및 잠재적인 병목 현상 파악
5. 베스트셀러 제품 카테고리 식별 및 수요 예측

## 3. Explore and Analyze Data

### 1. Most Popular Pages


#### 1-1 bounce_rate
- 자주 방문하는 페이지와 해당 페이지의 이탈률(bounce_rate) 확인
- `이탈률(bounce_rate) = 이탈 횟수 / 총 세션 수`, 
    - (이탈 횟수 = 사용자가 사이트에 들어왔다가 바로 떠난 세션)
- 이탈률이 높을 때의 추측
    - 사용자가 사이트에서 기대한 내용을 못 봤을 때 - 검색 엔진 최적화(SEO) 수행 필요
    - 페이지에 기능 또는 디스플레이 문제가 있을 때

In [13]:
# hit = 1

query = """
        SELECT
            hits.page.pagePath AS landing_page,
            COUNT(*) AS views,
            SUM(totals.bounces)/COUNT(*) AS bounce_rate
        FROM
            `bigquery-public-data.google_analytics_sample.ga_sessions_*`,
            UNNEST(hits) AS hits
        WHERE
            _TABLE_SUFFIX BETWEEN '20160801' AND '20170801'
        AND
            hits.type = 'PAGE'
        AND
            hits.hitNumber = 1
        GROUP BY
            landing_page
        ORDER BY
            views DESC
        LIMIT 10
        """
result = client.query(query).result().to_dataframe()
result.head(10)

,landing_page,views,bounce_rate
0,/home,612140,0.495475
1,/google+redesign/shop+by+brand/youtube,81512,0.562347
2,/google+redesign/apparel/men++s/men++s+t+shirts,20685,0.441141
3,/signin.html,16296,0.361622
4,/google+redesign/apparel/mens/mens+t+shirts,12691,0.475061
5,/basket.html,9431,0.454565
6,/google+redesign/drinkware,8833,0.533794
7,/google+redesign/bags,8608,0.451208
8,/google+redesign/apparel/men++s/men++s+outerwear,6345,0.432782
9,/google+redesign/apparel,6326,0.448150


- FROM
    - 샘플 데이터셋에서 ga_sessions_*이 파티션화된 테이블
    - `UNNEST(hits) AS hits` 중첩된 hits필드 펼치기 (반복 레코드 필드임)
- WHERE
    - `_TABLE_SUFFIX` 를 기반으로 20160801 ~ 20170801 데이터 필터링,
    - `hits.type` Page형식의 히트만 고려
- GROUP BY
    - `landing_page` 필드를 기반으로 데이터 그룹화
- SELECT
    - 해당되는 데이터의 `landing_page`, `views`, `bounce_rate`만 볼 것임(`AS`로 컬럼명 수정)
    - `landing_page` 페이지 경로(path)
    - `views` 카운트 COUNT(*)
    - `bounce_rate` 이탈률 = 이탈한 방문 수(totals.bounces의 합) / 총 방문 수(총 count)


💡 __Insight__  
- Men's `Apparel`과 `Bags` 다른 페이지에 비해 이탈률(bounce_rate)이 상대적으로 낮다.
- `youtube`, `drinkware` 조회수(views)는 많지만 이탈률(bounce_rate)이 높다. 


#### 1-2 exit rate
- exit rate는 랜딩 페이지뿐만 아니라 모든 페이지에 대해 계산
- `이탈률(exit_rate) = 이탈 수 / 총 세션 수`
    - (이탈 수 = 사용자가 페이지로 이동한 후 사이트를 떠난 세션)
- exit_rate이 높으면 사용자가 페이지로 이동할 때 예상한 내용이 표시되지 않거나 페이지 기능 또는 디스플레이에 문제가 있는 것

In [14]:
query = """
        SELECT
            hits.page.pagePath AS page,
            COUNT(*) AS views,
            SUM(totals.bounces)/COUNT(*) AS exit_rate
        FROM
            `bigquery-public-data.google_analytics_sample.ga_sessions_*`,
            UNNEST(hits) AS hits
        WHERE
            _TABLE_SUFFIX BETWEEN '20160801' AND '20170801'
        AND
            hits.type = 'PAGE'
        GROUP BY
            page
        ORDER BY
            views DESC
        LIMIT 10
        """
result = client.query(query).result().to_dataframe()
result.head(10)

,page,views,exit_rate
0,/home,981285,0.309920
1,/basket.html,209360,0.020524
2,/google+redesign/shop+by+brand/youtube,145026,0.316198
3,/signin.html,101299,0.058322
4,/store.html,93551,0.025505
5,/google+redesign/apparel/men++s/men++s+t+shirts,67471,0.135392
6,/asearch.html,62380,0.031677
7,/google+redesign/electronics,56839,0.047116
8,/google+redesign/apparel,56552,0.050272
9,/google+redesign/bags,53686,0.072458


__💡 Insight__
- 다른 페이지에 비해 남성 의류 관련 페이지의 exit_rate이 약간 높음
- youtube의 exit_rate이 매우 높음


#### 1-3 bounce_rate와 exit_rate의 차이 = 사용자가 처음 페이지를 방문한 순간의 정보 vs 모든 페이지 정보


```sql
# bounce_rate
 WHERE
            _TABLE_SUFFIX BETWEEN '20160801' AND '20170801'
        AND
            hits.type = 'PAGE'
        AND
            hits.hitNumber = 1 ⭐⭐⭐⭐⭐
            
# exit_rate
WHERE
            _TABLE_SUFFIX BETWEEN '20160801' AND '20170801'
        AND
            hits.type = 'PAGE'

```

### 2. Browser and Device
- 브라우저와 디바이스별로 세션과 호환성 문제 확인하기

In [15]:
query = """
        SELECT
            device.Browser AS browser,
            COUNT(*) AS sessions,
            SUM(totals.bounces)/COUNT(*) AS exit_rate
        FROM
            `bigquery-public-data.google_analytics_sample.ga_sessions_*`,
            UNNEST(hits) AS hits
        WHERE
            _TABLE_SUFFIX BETWEEN '20160801' AND '20170801'
        GROUP BY
            browser
        ORDER BY
            sessions DESC
        LIMIT 10
        """

result = client.query(query).result().to_dataframe()
result.head(10)

,browser,sessions,exit_rate
0,Chrome,3197849,0.091878
1,Safari,629906,0.157622
2,Firefox,133880,0.168195
3,Internet Explorer,62405,0.190369
4,Edge,38063,0.150934
5,Android Webview,25979,0.174872
6,Safari (in-app),19037,0.213532
7,Opera,15439,0.209988
8,Opera Mini,12767,0.314639
9,UC Browser,5807,0.283968


상위 5개 브라우저 중 Chrome의 이탈률은 9%로 비교적 낮은 반면,  
이탈률이 19%인 Internet Explorer를 제외한 나머지 브라우저는 Chrome에 비해 약 7~8% 정도 높음


### 3. Traffic Type and Quality
사이트로 유입되는 각 Traffic Type에 대한 세션 및 트랜잭션 데이터

In [16]:
query = """
        SELECT
            trafficSource.medium AS medium,
            COUNT(*) AS sessions,
            SUM(totals.bounces)/COUNT(*) AS exit_rate,
            SUM(totals.transactions) AS transactions,
            SUM(totals.totalTransactionRevenue)/1000000 AS total_revenue,
            SUM(totals.transactions)/COUNT(*) AS conversion_rate
        FROM
            `bigquery-public-data.google_analytics_sample.ga_sessions_*`,
            UNNEST(hits) AS hits
        WHERE
            _TABLE_SUFFIX BETWEEN '20160801' AND '20170801'
        GROUP BY
            medium
        ORDER BY
            sessions DESC
        """

result = client.query(query).result().to_dataframe()
result.head(10)

,medium,sessions,exit_rate,transactions,total_revenue,conversion_rate
0,(none),2246040,0.069684,363228,69842914.68,0.161719
1,organic,1141113,0.101857,97332,19504702.25,0.085296
2,referral,580309,0.285806,22040,3501224.27,0.037980
3,cpc,88591,0.055028,9457,1528330.72,0.106749
4,affiliate,55860,0.157698,334,29118.69,0.005979
5,cpm,41253,0.054372,5951,8676307.66,0.144256
6,(not set),509,0.108055,17,203.83,0.033399


__💡 Insight__
- traffic(medium)은 대부분 `organic`, `referral`인 것을 확인
- `cpc`와 `cpm`의 전환율(conversion_rate)이 다른 소스에 비해 상당히 높음
- 전환율(conversion_rate) = 트랜잭션이 완료된 세션의 비율


### 4. Identify Conversion Path and Possible Bottlenecks
- 고객의 전환 경로와 전환 과정에서 사용자가 사이트를 이탈하는 영역 식별

In [17]:
# action type별 총 조회수(hits) 집계

query = """
        SELECT
            CASE WHEN hits.eCommerceAction.action_type = '1' THEN 'Click through of product lists'
                 WHEN hits.eCommerceAction.action_type = '2' THEN 'Product detail views'
                 WHEN hits.eCommerceAction.action_type = '5' THEN 'Check out'
                 WHEN hits.eCommerceAction.action_type = '6' THEN 'Completed purchase'
            END AS action,
            COUNT(fullvisitorID) AS users,
        FROM
            `bigquery-public-data.google_analytics_sample.ga_sessions_*`,
            UNNEST(hits) AS hits,
            UNNEST(hits.product) AS product
        WHERE
            _TABLE_SUFFIX BETWEEN '20160801' AND '20170801'
        AND
            (
            hits.eCommerceAction.action_type != '0'
            AND
            hits.eCommerceAction.action_type != '3'
            AND
            hits.eCommerceAction.action_type != '4'
            )
        GROUP BY
            action
        ORDER BY
            users DESC
            """

result = client.query(query).result().to_dataframe()
result.head(10)

,action,users
0,Click through of product lists,445760
1,Product detail views,362607
2,Check out,248936
3,Completed purchase,74457


In [19]:
# funnel graph
import plotly.graph_objs as go
funnel_graph = go.Figure(go.Funnel(y = result['action'],
                                x = result['users'],
                                textposition = 'inside',  # Funnel 그래프 안에 텍스트를 표시할 위치를 'inside'로 설정
                                textinfo = 'value + percent initial'), # 각 단계의 값과 초기값의 백분율을 나타내도록 설정
                            layout = go.Layout(height = 400, width = 800)
                        )
funnel_graph.update_layout(title_text = 'Google Merchandise Store Conversion Path')

funnel_graph.show()

__💡 Insight__  
- 제품 세부 정보를 확인한 사용자의 69%가 결제 페이지로 이동  
- 그러나 이러한 사용자 중 실제로 구매를 완료한 비율은 30%에 불과하다는 것을 알 수 있음

### 5. Identify Best-Selling Product Categories and Forecast Demand
가장 인기 있는 제품 카테고리

In [21]:
query = """
        SELECT
            product.v2ProductCategory AS category,
            SUM(totals.transactions) AS transactions,
            SUM(totals.totalTransactionRevenue)/1000000 AS total_revenue
        FROM
            `bigquery-public-data.google_analytics_sample.ga_sessions_*`,
            UNNEST(hits) AS hits,
            UNNEST(hits.product) AS product
        WHERE
            _TABLE_SUFFIX BETWEEN '20160801' AND '20170801'
        GROUP BY
            category
        ORDER BY
            total_revenue DESC
        LIMIT 10
        """

cat_result = client.query(query).result().to_dataframe()
cat_result.head(10)

,category,transactions,total_revenue
0,(not set),155239,51801830.97
1,Apparel,133504,44923859.06
2,Home/Apparel/Men's/Men's-T-Shirts/,147131,25222312.08
3,Home/Office/,100994,20222511.87
4,Home/Electronics/,62603,13716689.08
5,Office,41837,12733564.97
6,Home/Apparel/,61127,11863650.76
7,Home/Apparel/Women's/Women's-T-Shirts/,58134,11448008.84
8,Home/Drinkware/Water Bottles and Tumblers/,44300,10600034.98
9,Home/Apparel/Men's/,77731,9781940.51


__💡insight__ 
- 가장 많이 팔리는 카테고리 중 일부는 `Apparel`와 관련이 있으며, `Men's-T-Shirts`가 가장 높은 순위
- `Office`, `Electronics`, `Water Bottles and Tumblers`도 잘 팔리고 있음
- `Apparel` 카테고리에서 가장 잘 팔리는 제품인 `Men's T-Shirts`의 수요를 예측하기 위해 7일간의 거래 이동 평균을 계산


In [ ]:
query = """
        WITH daily_mens_tshirt_transaction AS
        (
        SELECT 
        )

        """